<a href="https://colab.research.google.com/github/kenken0830/Graphing/blob/main/Demucs_Music_Source_Separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title #**Setup**

# install demucs
! pip install demucs

# clone github code
! git clone https://github.com/cedro3/demucs.git
%cd demucs

# import library
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO
from google.colab import files
import os
import shutil

# define function
def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

def reset_folder(path):
    if os.path.isdir(path):
      shutil.rmtree(path)
    os.makedirs(path,exist_ok=True)

# make directory
! mkdir target
! mkdir separated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 36.9 MB/s 
     |████████████████████████████████| 87 kB 7.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 393 kB 64.4 MB/s 
     |████████████████████████████████| 59 kB 9.2 MB/s 
     |████████████████████████████████| 190 kB 66.5 MB/s 
     |████████████████████████████████| 46 kB 5.1 MB/s 
     |████████████████████████████████| 73 kB 1.4 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 117 kB 52.0 MB/s 
  Created wheel for demucs: filename=demucs-3.0.5-py3-none-any.whl size=58966 sha256=9de3346351fa41e75a1f8d4bc4b060cd6a7781ea84f79624d5792a9770f49782
  Stored in directory: /root/.cache/pip/wheels/0e/07/ce/3ac3936612565655cddff6f39f6b3db4f85dde2b0edca9d2c9
  Crea

Cloning into 'demucs'...
remote: Enumerating objects: 5373, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 5373 (delta 94), reused 136 (delta 77), pack-reused 5195
Receiving objects: 100% (5373/5373), 85.89 MiB | 45.50 MiB/s, done.
Resolving deltas: 100% (824/824), done.
/content/demucs


In [5]:
#@title #**Separate music**

#@markdown **please upload mp3 to music folder**

# setting
input = 'sound (1).mp3' #@param {type:"string"}
reset_folder('target')
reset_folder('separated')
shutil.copy('./music/'+input, './target/'+input)
karaoke = False #@param {type:"boolean"}

# Customize the following options!
model = "mdx_extra"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.

if karaoke:
  two_stems = "vocals"
else:
  two_stems = None   # only separate one stems from the rest, for instance

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = './target/'
out_path = './separated/'

separate()

Going to separate the files:
target/sound (1).mp3
With command:  python3 -m demucs.separate -o ./separated/ -n mdx_extra --mp3 --mp3-bitrate=320
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/demucs/separated/mdx_extra
Separating track target/sound (1).mp3


100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:01<00:00, 21.29seconds/s]
100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 51.68seconds/s]
100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 51.32seconds/s]
100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:00<00:00, 53.68seconds/s]


In [6]:
#@title #**Play separated music**

from IPython.display import Audio
#@markdown　-If karaoka, you can select vocals or no_vocals\
#@markdown -If not karaoke, you can select bass, drums, other, vocals

part ="vocals"#@param ["bass", "drums", "other", "vocals", "no_vocals"]
file_path = './separated/mdx_extra/'+os.path.splitext(input)[0]+'/'+part+'.mp3'
Audio(file_path)

In [7]:
#@title #**Download separated music**
from google.colab import files

if os.path.isfile('separated.zip'):
  os.remove('separated.zip')

! zip -r separated.zip separated/mdx_extra
files.download('separated.zip')

  adding: separated/mdx_extra/ (stored 0%)
  adding: separated/mdx_extra/sound (1)/ (stored 0%)
  adding: separated/mdx_extra/sound (1)/other.mp3 (deflated 2%)
  adding: separated/mdx_extra/sound (1)/drums.mp3 (deflated 1%)
  adding: separated/mdx_extra/sound (1)/vocals.mp3 (deflated 2%)
  adding: separated/mdx_extra/sound (1)/bass.mp3 (deflated 1%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>